In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("train.csv" )
data=np.array(data)
X_train1=data[0:5804,9:69]
X_train2=data[0:5804,0:9]
y_train1=data[0:5804,69:129]
y_train2=data[0:5804,129:189]
y_train3=data[0:5804,189:249]

data2 = pd.read_csv("test.csv" )
data2=np.array(data2)
X_test1=data2[0:1451,9:69]
X_test2=data2[0:1451,0:9]
y_test1=data2[0:1451,69:129]
y_test2=data2[0:1451,129:189]
y_test3=data2[0:1451,189:249]

# print(X_train2)
print(X_train1.shape,X_train2.shape,y_train1.shape,y_train2.shape,y_train3.shape,X_test1.shape,X_test2.shape,y_test1.shape,y_test2.shape,y_test3.shape)

(5804, 60) (5804, 9) (5804, 60) (5804, 60) (5804, 60) (1451, 60) (1451, 9) (1451, 60) (1451, 60) (1451, 60)


In [3]:
from keras import backend as K
class Self_Attention(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Self_Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', shape=(3,input_shape[2], self.output_dim), initializer='uniform', trainable=True)
        super(Self_Attention, self).build(input_shape)

    def call(self, x):
        WQ = K.dot(x, self.kernel[0])
        WK = K.dot(x, self.kernel[1])
        WV = K.dot(x, self.kernel[2])

        attn_scores = K.batch_dot(WQ, K.permute_dimensions(WK, [0, 2, 1]))
        attn_scores = attn_scores / (64**0.5)
        attn_scores= K.softmax(attn_scores)

        V = K.batch_dot(attn_scores, WV)
        return K.identity(V),attn_scores

    def compute_output_shape(self, input_shape):
        return (input_shape[0],input_shape[1],self.output_dim)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            "output_dim" : self.output_dim
        })
        return config
    
def conv_unit(unit, input_layer):
    layer = tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu')(input_layer)
    layer = tf.keras.layers.BatchNormalization()(layer) 
    layer = tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding='same', activation=None)(layer )
    layer = tf.keras.layers.BatchNormalization()(layer) 
    layer = tf.keras.layers.Add()([layer, input_layer])
    layer = tf.keras.layers.Activation("relu")(layer)
    layer = tf.keras.layers.MaxPooling1D(2)(layer)
    return layer

In [4]:
from keras.layers import Input
from keras.models import Model
from keras.utils import plot_model
from keras import regularizers
import seaborn as sns

input1 = Input(shape=(60,1))
layer1 = tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1)(input1)
for i in range(3):
        layer1 = conv_unit(i + 1,layer1)

In [8]:
layer1 = tf.keras.layers.Flatten()(layer1)
layer1 = tf.keras.layers.Dense(128,activation='relu')(layer1)
layer1 = tf.keras.layers.Dense(16,activation='relu')(layer1)
input2 = Input(shape=(9,1))
conc4 = tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1)(input2)
conc4,attn_scores = Self_Attention(16)(conc4)
conc4 = tf.keras.layers.Reshape((80,))(conc4)
conc4 = tf.keras.layers.Dense(128,activation='relu')(conc4)
conc4 = tf.keras.layers.Dense(128,activation='relu')(conc4)
conc4 = tf.keras.layers.Dense(128,activation='relu')(conc4)
conc4 = tf.keras.layers.Concatenate()([conc4,layer1])
conc4 = tf.keras.layers.Dense(128,activation='relu')(conc4)
conc4 = tf.keras.layers.Dense(128,activation='relu')(conc4)
outputs1 = tf.keras.layers.Dense(units=60, name='outputs1')(conc4)
outputs2 = tf.keras.layers.Dense(units=60, name='outputs2')(conc4)
outputs3 = tf.keras.layers.Dense(units=60, name='outputs3')(conc4)
model = Model(inputs=[input1,input2],outputs=[outputs1,outputs2,outputs3])
plot_model(model, to_file='FRSNet.png',show_shapes='true')
print(model.summary())
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mae',metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60, 1)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 56, 32)       192         ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 56, 32)       5152        ['conv1d[0][0]']                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 56, 32)      128         ['conv1d_1[0][0]']               
 alization)                                                                                 

In [ ]:
history = model.fit([X_train1,X_train2],
                    [y_train1,y_train2,y_train3],
                    batch_size=32,
                    epochs=3000,
                    validation_data=([X_test1,X_test2],[y_test1,y_test2,y_test3]),
                    verbose=2)

In [ ]:
from matplotlib import rcParams
from matplotlib.pyplot import MultipleLocator
config = {
    "font.family":"Time New Roman",
    "font.size":18,
}
history_dict=history.history
history_dict.keys()
loss_values=history_dict['loss']
val_loss_values=history_dict['val_loss']
epochs=range(1,len(loss_values)+1)
plt.plot(epochs,loss_values,'b',color='blue',linestyle='--',label='Training loss')
plt.plot(epochs,val_loss_values,'b',color='red',label='val_loss_values')
plt.title('Traing and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
ax=plt.gca()
x_major_locator=MultipleLocator(500)
ax.xaxis.set_major_locator(x_major_locator)
fig=plt.gcf()
fig.set_size_inches(15,7)
rcParams.update(config)
plt.show()

In [ ]:
loss=pd.DataFrame(history_dict['loss'])
val_loss=pd.DataFrame(history_dict['val_loss'])

with pd.ExcelWriter('loss.xlsx') as writer:
    loss.to_excel(writer,sheet_name='loss', index=False,header=False)
    val_loss.to_excel(writer,sheet_name='val_loss', index=False,header=False)

In [ ]:
model.save('B-c-2.h5')

In [ ]:
[pred_x1,pred_x2,pred_x3]=model.predict([X_test1,X_test2])
print(pred_x1.shape,pred_x2.shape,pred_x3.shape)
pred_x1=pd.DataFrame(pred_x1)
pred_x2=pd.DataFrame(pred_x2)
pred_x3=pd.DataFrame(pred_x3)

with pd.ExcelWriter('B-c-2test.xlsx') as writer:
    pred_x1.to_excel(writer,sheet_name='Sa', index=False,header=False)
    pred_x2.to_excel(writer,sheet_name='Sv', index=False,header=False)
    pred_x3.to_excel(writer,sheet_name='Sd', index=False,header=False)

In [ ]:
[pred_x1,pred_x2,pred_x3]=model.predict([X_train1,X_train2])
print(pred_x1.shape,pred_x2.shape,pred_x3.shape)
pred_x1=pd.DataFrame(pred_x1)
pred_x2=pd.DataFrame(pred_x2)
pred_x3=pd.DataFrame(pred_x3)

with pd.ExcelWriter('B-c-2train.xlsx') as writer:
    pred_x1.to_excel(writer,sheet_name='Sa', index=False,header=False)
    pred_x2.to_excel(writer,sheet_name='Sv', index=False,header=False)
    pred_x3.to_excel(writer,sheet_name='Sd', index=False,header=False)

In [ ]:
model = tf.keras.models.load_model("B-c-2.h5",custom_objects={'Self_Attention':Self_Attention},compile=False)